# JSON - Quick Start Guide

JSON is very similar to XML; It is a form of text notation designed for data exchange. JSON has one big advantage over XML; it is significantly more compact than XML. Whilst JSON is not a new technology, the inclusion of tools to handle JSON in SQL Server were only introduced in SQL 2016.

## The Basics of JSON

Below is the sample of JSON that I will be using in all of my examples:

![JSON](./Images/json_orig.png "JSON")

JSON is made up of objects, which are collections of key value pairs encased in a curly braces, { and }. Each key value pair is formatted as a key, in double quotes, followed by a colon, followed by the value. Values can be strings, numbers, Boolean values, Null values, other objects, or arrays. Arrays are collections of objects or values encased on square brackets and separated by commas.

JSON represents a data hierarchy, whereby everything within an object belongs to that instance of the object. For example, in the above example there are 2 objects within the teams array, each containing an array of players that belong specifically to that team.

## JSON in SQL

Unlike XML; JSON does not have its own datatype within SQL Server, instead JSON is treated as an nvarchar(max). This means that, unlike XML, the validity of JSON within a string is not automatically enforced. Thankfully Microsoft have provided a function to validate JSON; ISJSON, which returns a 1 for valid and 0 for invalid. 

Give this a try:


In [ ]:
DECLARE @json NVARCHAR(4000)
SET @json = 
N'{
   "teams":[{"name":"Bradford Bulls",
                       "Players":[{"number":1, "name":"Lee Smith", "position":"Centre"},
                                            {"number":2, "name":"Ethan Ryan", "position":"Winger"} 
                                            ]
                     },
                    {"name":"Keighley Cougars",
                     "Players":[{"number":1, "name":"Ritchie Hawkyard", "position":"Fullback"},
                                           {"number":2, "name":"Andy Gabriel", "position":"Winger"} 
                                         ]
                  }]
  }'

Select ISJSON(@json)

SSMS does not currently offer formatting of JSON results, as it does with XML, and so I am using Visual Studio Code to format JSON samples.

There are a number of functions for interacting with JSON, but I'm going to concentrate on just one; OPENJSON.

## Querying JSON with OPENJSON

OPENJSON is a table value function taking 2 parameters; a JSON expression, and an optional path. Without a specified path OPENJSON will return the key, value and type of all top level objects:

```
​Select *
from OpenJSON(@json)
```

![JSON](./Images/openjson-1_orig.png "JSON")

Give this a go:


In [ ]:
DECLARE @json NVARCHAR(4000)
SET @json = 
N'{
   "teams":[{"name":"Bradford Bulls",
                       "Players":[{"number":1, "name":"Lee Smith", "position":"Centre"},
                                            {"number":2, "name":"Ethan Ryan", "position":"Winger"} 
                                            ]
                     },
                    {"name":"Keighley Cougars",
                     "Players":[{"number":1, "name":"Ritchie Hawkyard", "position":"Fullback"},
                                           {"number":2, "name":"Andy Gabriel", "position":"Winger"} 
                                         ]
                  }]
  }'

Select *
from OpenJSON(@json)

The Type values are:​
  0. Null
  1. String
  2. Int
  3. Boolean - True/False
  4. Array
​  5. Object

Specifying a path parameter allows you to specify which part of the JSON you are querying. JSON paths are similar to XML paths, in that they are the sequence of objects through which you need to navigate to get to the object that you are interested in. Unlike XML the path is separated by periods (.) rather than slashes. When querying an array you need to specify which object within the array you are referencing by specifying the position number of the relevant item in square brackets. Array positions are zero based, so the first item will be position 0, the second position 1, etc. The following example returns all data from the second player, from the second team:
​
```
Select *
from OpenJSON(@json, '$.teams[1].Players[1]')
```

![JSON](./Images/openjson-2_orig.png "JSON")


Give this a go:

In [ ]:
DECLARE @json NVARCHAR(4000)
SET @json = 
N'{
   "teams":[{"name":"Bradford Bulls",
                       "Players":[{"number":1, "name":"Lee Smith", "position":"Centre"},
                                            {"number":2, "name":"Ethan Ryan", "position":"Winger"} 
                                            ]
                     },
                    {"name":"Keighley Cougars",
                     "Players":[{"number":1, "name":"Ritchie Hawkyard", "position":"Fullback"},
                                           {"number":2, "name":"Andy Gabriel", "position":"Winger"} 
                                         ]
                  }]
  }'

Select *
from OpenJSON(@json, '$.teams[1].Players[1]')

Whilst the above queries are fine for visualising data from JSON, you are generally not interested in the key and type of your data. What you want is a query akin to a query against a table. A query where each value is held within its own field. Adding a WITH statement allows us to define those fields...

```
​Select  name, number, position
from    OpenJSON(@json,'$.teams[1].Players[1]')
With    (
        number int,
        name varchar(50),
        position varchar(50)
        )
```

![JSON](./Images/openjson-3_orig.png "JSON")

Give this a go:


In [ ]:
DECLARE @json NVARCHAR(4000)
SET @json = 
N'{
   "teams":[{"name":"Bradford Bulls",
                       "Players":[{"number":1, "name":"Lee Smith", "position":"Centre"},
                                            {"number":2, "name":"Ethan Ryan", "position":"Winger"} 
                                            ]
                     },
                    {"name":"Keighley Cougars",
                     "Players":[{"number":1, "name":"Ritchie Hawkyard", "position":"Fullback"},
                                           {"number":2, "name":"Andy Gabriel", "position":"Winger"} 
                                         ]
                  }]
  }'

​Select  name, number, position
from    OpenJSON(@json,'$.teams[1].Players[1]')
With   (
        number int,
        name varchar(50),
        position varchar(50)
        )

By default the field names that you specify in the WITH statement have to match the key values from the JSON. However; by specifying a JSON path within the field definitions you can specify which key value you want to return. The following example returns the data relating to the first player from the first team, but specifies JSON paths to allow renaming of the resulting fields:

```
Select  PlayerName, SquadNumber, Position
from    OpenJSON(@json,'$.teams[0].Players[0]')
With    (
        SquadNumber int '$.number',
        PlayerName varchar(50) '$.name',
        Position varchar(50) '$.position'
        )
```

![JSON](./Images/openjson-4_orig.png "JSON")

Give this a go:

In [ ]:
DECLARE @json NVARCHAR(4000)
SET @json = 
N'{
   "teams":[{"name":"Bradford Bulls",
                       "Players":[{"number":1, "name":"Lee Smith", "position":"Centre"},
                                            {"number":2, "name":"Ethan Ryan", "position":"Winger"} 
                                            ]
                     },
                    {"name":"Keighley Cougars",
                     "Players":[{"number":1, "name":"Ritchie Hawkyard", "position":"Fullback"},
                                           {"number":2, "name":"Andy Gabriel", "position":"Winger"} 
                                         ]
                  }]
  }'

Select  PlayerName, SquadNumber, Position
from    OpenJSON(@json,'$.teams[0].Players[0]')
With    (
        SquadNumber int '$.number',
        PlayerName varchar(50) '$.name',
        Position varchar(50) '$.position'
        )


Obviously, this method of extracting data is very limited, and would be unworkable if the JSON contained 100 players rather than just 4. Thankfully there is another method which allows you to query entire arrays.

The following example lists all players form all teams:

```
​Select TeamName, SquadNumber, PlayerName, position
from OpenJSON(@json,'$.teams')
With (
     TeamName varchar(50) '$.name',
     Players NVARCHAR(max) As JSON
     )
cross apply OpenJSON(Players)
With (
     SquadNumber int '$.number',
     PlayerName varchar(50) '$.name',
     position varchar(50)
     )
```

![JSON](./Images/openjson-5_orig.png "JSON")

Give this a go:

In [ ]:
DECLARE @json NVARCHAR(4000)
SET @json = 
N'{
   "teams":[{"name":"Bradford Bulls",
                       "Players":[{"number":1, "name":"Lee Smith", "position":"Centre"},
                                            {"number":2, "name":"Ethan Ryan", "position":"Winger"} 
                                            ]
                     },
                    {"name":"Keighley Cougars",
                     "Players":[{"number":1, "name":"Ritchie Hawkyard", "position":"Fullback"},
                                           {"number":2, "name":"Andy Gabriel", "position":"Winger"} 
                                         ]
                  }]
  }'

​Select TeamName, SquadNumber, PlayerName, position
from OpenJSON(@json,'$.teams')
With (
     TeamName varchar(50) '$.name',
     Players NVARCHAR(max) As JSON
     )
cross apply OpenJSON(Players)
With (
     SquadNumber int '$.number',
     PlayerName varchar(50) '$.name',
     position varchar(50)
     )

In this example we have one OPENJSON statement which exposes the teams array, and defines 2 fields; the team name, and the players array. The players array is defined as an nvarchar(max) data type, and the addition of the As JSON clause allows us to query this field with a second OPENJSON statement. The second OPENJSON statement exposes the contents of every object within the Player array.

The following example demonstrates querying JSON from  table rather than a variable:

```
Declare @SquadsTable table ( TestJSON varchar(max))

insert @SquadsTable (TestJSON)
values (@json)

Select  TeamName, number, PlayerName, position
from    @SquadsTable
cross apply OpenJSON(TestJSON,'$.teams')
With    (
        TeamName varchar(50) '$.name', 
        Players NVARCHAR(max) As JSON
        )
cross apply OpenJSON(Players)
With    (
        number int,
        PlayerName varchar(50) '$.name',
        position varchar(50)
        )
```

The results of this query are identical to the prior query.

In [ ]:
Declare @SquadsTable table ( TestJSON varchar(max))

DECLARE @json NVARCHAR(4000)
SET @json = 
N'{
   "teams":[{"name":"Bradford Bulls",
                       "Players":[{"number":1, "name":"Lee Smith", "position":"Centre"},
                                            {"number":2, "name":"Ethan Ryan", "position":"Winger"} 
                                            ]
                     },
                    {"name":"Keighley Cougars",
                     "Players":[{"number":1, "name":"Ritchie Hawkyard", "position":"Fullback"},
                                           {"number":2, "name":"Andy Gabriel", "position":"Winger"} 
                                         ]
                  }]
  }'

insert @SquadsTable (TestJSON)
values (@json)

Select  TeamName, number, PlayerName, position
from    @SquadsTable
cross apply OpenJSON(TestJSON,'$.teams')
With    (
        TeamName varchar(50) '$.name', 
        Players NVARCHAR(max) As JSON
        )
cross apply OpenJSON(Players)
With    (
        number int,
        PlayerName varchar(50) '$.name',
        position varchar(50)
        )

JSON supports arrays of values, in addition to arrays of objects. The following example demonstrates querying an array of values:

```
​Select value PlayerName
FROM OPENJSON(N'{"Players":["Lee Smith", "Ethan Ryan", "Ritchie Hawkyard", "Andy Gabriel"]}','$.Players')
```

![JSON](./Images/openjson-6_orig.png "JSON")

Give this a go:

In [ ]:
​Select  value PlayerName
FROM    OPENJSON(N'{"Players":["Lee Smith", "Ethan Ryan", "Ritchie Hawkyard", "Andy Gabriel"]}','$.Players')

## Generating JSON Result Sets

As well was storing and querying JSON, SQL Server can generate JSON from result sets. You do this by adding the FOR JSON keywords to the end of your query. For the demonstration I will use the following table as my source:

```
Create table #Squad
(
TeamName varchar(50),
SquadNumber int,
PlayerName varchar(50),
Position varchar(50)
)

insert #Squad
values  ('Bradford Bulls',1,'Lee Smith','Centre'),
        ('Bradford Bulls',2,'Ethan Ryan','Winger'),
        ('Keighley Cougars',1,'Ritchie Hawkyard','Fullback'),
        ('Keighley Cougars',2,'Andy Gabriel','Winger')
```

The following code demonstrates creating a simple JSON document:

```
Select  *
from    #Squad
FOR JSON Auto
```

![JSON](./Images/for-json-2_orig.png "JSON")

Give this a go:

In [ ]:
Create table #Squad
(
TeamName varchar(50),
SquadNumber int,
PlayerName varchar(50),
Position varchar(50)
)

insert #Squad
values  ('Bradford Bulls',1,'Lee Smith','Centre'),
        ('Bradford Bulls',2,'Ethan Ryan','Winger'),
        ('Keighley Cougars',1,'Ritchie Hawkyard','Fullback'),
        ('Keighley Cougars',2,'Andy Gabriel','Winger')

Select  *
from    #Squad
FOR JSON Auto
;

Drop Table #Squad
;

When the Auto keyword is specified you will have no control over the formatting of the JSON. The JSON will be created as a single array of objects, with no hierarchy.

The following example uses the Path keyword instead of Auto. This allows you far more control over the formatting of the JSON. Firstly I am specifying a root object, teams, to contain our array:

```
Select  *
from    #Squad
FOR JSON Path, Root('teams')
```

![JSON](./Images/for-json-1_orig.png "JSON")


In [ ]:
Create table #Squad
(
TeamName varchar(50),
SquadNumber int,
PlayerName varchar(50),
Position varchar(50)
)

insert #Squad
values  ('Bradford Bulls',1,'Lee Smith','Centre'),
        ('Bradford Bulls',2,'Ethan Ryan','Winger'),
        ('Keighley Cougars',1,'Ritchie Hawkyard','Fullback'),
        ('Keighley Cougars',2,'Andy Gabriel','Winger')
;

Select  *
from    #Squad
FOR JSON Path, Root('teams')
;

Drop Table #Squad
;

To produce the JSON from my original sample we need to create nested JSON. Nesting involves defining correlated sub queries to  define each array.

```
Select  TeamName as 'name',
        (            
        Select  SquadNumber as 'number',
                PlayerName as 'name',
                Position as 'position'
        from    #Squad p
        where   TeamName = s.TeamName
        for JSON Path
        ) Players
from    (
        Select  distinct  TeamName
        from    #Squad
        ) s
for JSON Path, Root('teams')
```

In this example I want each team to appear only once rather than be duplicated, so I am using an inline view to start with a distinct list of teams. This list has a defined Root (teams). Within each team we have a sub-query which defines an array of players.

![JSON](./Images/json-1_orig.png "JSON")


In [ ]:
Create table #Squad
(
TeamName varchar(50),
SquadNumber int,
PlayerName varchar(50),
Position varchar(50)
)

insert #Squad
values  ('Bradford Bulls',1,'Lee Smith','Centre'),
        ('Bradford Bulls',2,'Ethan Ryan','Winger'),
        ('Keighley Cougars',1,'Ritchie Hawkyard','Fullback'),
        ('Keighley Cougars',2,'Andy Gabriel','Winger')
;

Select  TeamName as 'name',
        (            
        Select  SquadNumber as 'number',
                PlayerName as 'name',
                Position as 'position'
        from    #Squad p
        where   TeamName = s.TeamName
        for JSON Path
        ) Players
from    (
        Select  distinct  TeamName
        from    #Squad
        ) s
for JSON Path, Root('teams')
;

Drop Table #Squad
;
